In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime as datetime
import matplotlib as mpl
plt.style.use(['ggplot'])
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
df=pd.read_csv("../input/montreal-bike-lanes/comptagesvelo2015.csv")

In [ ]:
df.head()

We are gonna change the date format and extract the day,month,dayofweeek.

In [ ]:
def change_date_format(data):
    return datetime.datetime.strptime(data,"%d/%m/%Y")

In [ ]:
df["dates"]=df["Date"].apply(change_date_format)

In [ ]:
df["month"]=df["dates"].dt.month
df["day"]=df["dates"].dt.day
df["dow"]=df["dates"].dt.dayofweek

In [ ]:
sns.heatmap(df.isnull())

There are four streets which are operated for sometime and closed and reopened after the year 

In [ ]:
mon=["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov"]
plt.figure(figsize=(14,7))
plt.plot(df["Date"],df["Maisonneuve_1"],label="Maisonneuve_1")
plt.plot(df["Date"],df['Parc U-Zelt Test'],label='Parc U-Zelt Test')
plt.plot(df["Date"],df['Pont_Jacques_Cartier'],label='Pont_Jacques_Cartier')
plt.plot(df["Date"],df['Saint-Laurent U-Zelt Test'],label='Saint-Laurent U-Zelt Test')
plt.xticks(np.arange(1,319,30),mon)
plt.legend(loc=2,bbox_to_anchor=(1.05,1))

In [ ]:
street_cols=df.columns[2:23]

In [ ]:
len(street_cols)

In [ ]:
street_cols

In [ ]:
fig = plt.figure(figsize=(14, 26))
columns = 3
rows = 7
ax = []
for i in range(columns*rows):
    ax.append( fig.add_subplot(rows, columns, i+1) )
    ax[i].set_title(street_cols[i])
    plt.setp(ax,xticks=np.arange(1,319,30), xticklabels=mon)
for i in range(len(street_cols)):
    ax[i].plot(df["Date"],df[street_cols[i]],label=street_cols)

    

    

It is evident thet the bike riders are lesser during winter and are normally disturbuted during the summer.

'Maisonneuve_3' street have lower bikers.But clearly shows hike in july.Maybe due to some parade or any festivals.

In [ ]:
df[df["Maisonneuve_3"]==df["Maisonneuve_3"].max()]

In [ ]:
df.fillna(value=0,inplace=True)

In [ ]:
val=[]
for i in range(len(df)):
    val.append(df.iloc[i,2:23].values.sum())
    

In [ ]:
df["total"]=val

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df["Date"],df["total"])
plt.xticks(np.arange(1,319,30),mon)

Highest number of bike in the city was on 29/05/2015

In [ ]:
df[df["total"]==df["total"].max()]

In [ ]:
data=df[["month","dow","day","total"]]

In [ ]:
plt.figure(figsize=(14,7))
sns.heatmap(data.corr(),annot=True)

month seems to be only correlated factor

# Scikit learn

In [ ]:
data=df[["month","dow","day","total"]]

In [ ]:
X=data.drop("total",axis=1)
y=data["total"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)


In [ ]:
lr=LinearRegression()

In [ ]:
lr.fit(X_train,y_train)

In [ ]:
lr.score(X_test,y_test)

In [ ]:
pred=lr.predict(X_test)

In [ ]:
plt.scatter(y_test,pred)

model doesn't firt very well.Because,there is not enough information.So,we can try adaboost regressor.

In [ ]:
abr=AdaBoostRegressor(n_estimators=300)

In [ ]:
abr.fit(X_train,y_train)

In [ ]:
abr.score(X_test,y_test)

In [ ]:
predictions=abr.predict(X_test)

In [ ]:
plt.scatter(y_test,predictions)

Adaboost regressor performs better than the linear regression.Still we have room to improve.We take in the weather 2015 data.

In [ ]:
weather=pd.read_csv("../input/weather-montreal-2015-en/eng-daily-01012015-12312015.csv")

In [ ]:
weather.columns

In [ ]:
plt.figure(figsize=(14,7))
sns.heatmap(weather.isnull())

We take only useful infromation and merge both dataframe.

In [ ]:
weathers=weather[['Mean Temp (°C)','Total Rain (mm)','Total Snow (cm)','Total Precip (mm)','Snow on Grnd (cm)','Spd of Max Gust (km/h)']]

In [ ]:
final_data=data.merge(weathers,left_index=True,right_index=True)

In [ ]:
final_data.fillna(0,inplace=True)

In [ ]:
sns.heatmap(final_data.isnull())

In [ ]:
plt.figure(figsize=(14,7))
sns.heatmap(final_data.corr(),annot=True)

We see month and mean temp positively corelated and snow on ground is neagtively corelated.

In [ ]:
data=final_data.iloc[1]
types=[]
for i in range(len(final_data.columns)):
    types.append(type(data[i]))

In [ ]:
columnss=pd.DataFrame(data=types,index=final_data.columns)

In [ ]:
columnss

Spd of Max Gust is of class string and has lot of comparison vectors.

In [ ]:
def change(data):
    if data=="<31":
        return 31
    else :
        return int(data)

In [ ]:
final_data["Spd of Max Gust (km/h)"]=final_data["Spd of Max Gust (km/h)"].apply(change)

In [ ]:
X=final_data.drop("total",axis=1)
y=final_data["total"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)


In [ ]:
lr.fit(X_train,y_train)

In [ ]:
lr.score(X_test,y_test)

In [ ]:
pred=lr.predict(X_test)

In [ ]:
plt.scatter(y_test,pred)

In [ ]:
abr=AdaBoostRegressor(n_estimators=300)

In [ ]:
abr.fit(X_train,y_train)

In [ ]:
abr.score(X_test,y_test)

In [ ]:
predictions=abr.predict(X_test)

In [ ]:
plt.scatter(y_test,predictions)

We see both linear regression and Adaboost regressor have increased score compared with without weather data.It can futher improve with festival data.When there is no rain,snow model finds it hard to predict.